

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_MULTILABEL_TOXIC.ipynb)




# **Detect toxic content in comments**

## 1. Colab setup

In [ ]:
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp

In [ ]:
import os
import json
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

# Start Spark session
spark = sparknlp.start(gpu=True)

## 2. Pipeline creation and training

Create pipeline to be trained on example inputs.

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = UniversalSentenceEncoder.pretrained(lang='en') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

classsifierdl = MultiClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("labels") \
    .setMaxEpochs(10) \
    .setLr(1e-3) \
    .setThreshold(0.7) \
    .setValidationSplit(0.2) \
    .setOutputLogsPath('./') \
    .setEnableOutputLogs(True)

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    embeddingsSentence,
    classsifierdl
])

Download training and testing datasets.

In [ ]:
! curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_train.snappy.parquet'
! curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_test.snappy.parquet'
trainDataset = spark.read.parquet("toxic_train.snappy.parquet")
testDataset = spark.read.parquet("toxic_test.snappy.parquet")

In [ ]:
res = trainDataset.toPandas()

In [ ]:
res

In [ ]:
all_labs = []
for r in res['labels'].values:
    all_labs.extend(r)
set(all_labs)

Train the pipeline model on the training dataset.

In [ ]:
pipelineModel = pipeline.fit(trainDataset)

Example training log:


```
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 64 - training_examples: 127657 - classes: 7
Epoch 0/10 - 46.00s - loss: 0.079907365 - acc: 0.9723665 - val_loss: 0.07093345 - val_acc: 0.97352755 - val_f1: 0.91585624 - val_tpr: 0.9008827 - batches: 1995
Epoch 1/10 - 28.26s - loss: 0.06964213 - acc: 0.9747156 - val_loss: 0.069492534 - val_acc: 0.9739863 - val_f1: 0.91734964 - val_tpr: 0.9029063 - batches: 1995
Epoch 2/10 - 27.99s - loss: 0.06808146 - acc: 0.9752242 - val_loss: 0.06841504 - val_acc: 0.97444665 - val_f1: 0.9187962 - val_tpr: 0.9039833 - batches: 1995
Epoch 3/10 - 27.94s - loss: 0.066884466 - acc: 0.9757066 - val_loss: 0.06769186 - val_acc: 0.9746787 - val_f1: 0.9195223 - val_tpr: 0.90438014 - batches: 1995
Epoch 4/10 - 28.04s - loss: 0.06587076 - acc: 0.9761073 - val_loss: 0.067252316 - val_acc: 0.9749198 - val_f1: 0.92028916 - val_tpr: 0.9049118 - batches: 1995
Epoch 5/10 - 28.05s - loss: 0.06501821 - acc: 0.97637606 - val_loss: 0.06700255 - val_acc: 0.9750182 - val_f1: 0.9205762 - val_tpr: 0.90496385 - batches: 1995
Epoch 6/10 - 27.92s - loss: 0.064287946 - acc: 0.9765889 - val_loss: 0.06686394 - val_acc: 0.9750045 - val_f1: 0.9205367 - val_tpr: 0.904986 - batches: 1995
Epoch 7/10 - 28.19s - loss: 0.063645855 - acc: 0.97682655 - val_loss: 0.0667824 - val_acc: 0.97498673 - val_f1: 0.92048156 - val_tpr: 0.90499973 - batches: 1995
Epoch 8/10 - 28.53s - loss: 0.06306508 - acc: 0.97704613 - val_loss: 0.06673989 - val_acc: 0.974991 - val_f1: 0.9205486 - val_tpr: 0.9056079 - batches: 1995
Epoch 9/10 - 28.61s - loss: 0.0625258 - acc: 0.97720623 - val_loss: 0.06672904 - val_acc: 0.9750411 - val_f1: 0.9207759 - val_tpr: 0.90621996 - batches: 1995
```



## 3. Testing and examples

Run the model on the test dataset to evaluate performance and generate examples.

In [ ]:
test_res = pipelineModel.transform(testDataset)

Visualizing the raw test dataset after classification.

In [ ]:
test_res.show()

Write some sample inputs and outputs from the test dataset to file.

In [ ]:
! mkdir -p inputs
! mkdir -p outputs

result = test_res.toPandas()
for i in range(1, 11):
    text = result[['document']].iloc[i][0][0].result
    with open(f'inputs/{i}.txt', 'w') as f:
        f.write(text[:96].replace('\n', '') + " ...\n" + text)
    result[['class']].iloc[i].to_json(f'outputs/{i}.json')